# Matrix Multiplication with PyOpenCL

This notebook demonstrates how to perform matrix multiplication using PyOpenCL. The OpenCL kernel implementation is left empty for you to complete as an exercise.

In [ ]:
import sys
import os

import pyopencl as cl
import numpy as np

sys.path.append("..")
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

from helpers import profile_gpu

%load_ext pyopencl.ipython_ext

Create context and queue.


In [ ]:
platform = cl.get_platforms()[0]

ctx = cl.Context(
    dev_type=cl.device_type.ALL, 
    properties=[(cl.context_properties.PLATFORM, platform)])    

queue = cl.CommandQueue(ctx, properties=cl.command_queue_properties.PROFILING_ENABLE)
    
devices = ctx.get_info(cl.context_info.DEVICES)
for d in devices:
    print(f"device={d}")

## Define Matrices

Let's define two matrices to multiply. You can change their size and values as needed.

In [ ]:
# Define two matrices A and B
N = 1024
A = np.random.randn(N, N).astype(np.float32)
B = np.random.randn(N, N).astype(np.float32)


## Perform Matrix Multiplication on the GPU

We will multiply matrices A and B using an OpenCL kernel. Implement the kernel code to perform standard matrix multiplication as done in algebra. For two matrices A (of size m×n) and B (of size n×p), their product C = AB is an m×p matrix where each element C[i, j] is computed as the sum of products of the i-th row of A and the j-th column of B:

$$
C_{i,j} = \sum_{k=1}^{N} A_{i,k} \cdot B_{k,j}
$$


This operation is also known as "GEMM" (General Matrix Multiply) in numerical computing libraries.

We will build-up the optimal solution in stages. Start simple, and implement the multiplication using just global memory, where each work item computes output value for one cell in the output matrix.

You can assume that all matrices are square for now, if you find it more convenient.

In [ ]:
# Set up OpenCL context and buffers
mf = cl.mem_flags
d_a = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=A)
d_b = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=B)
d_c = cl.Buffer(ctx, mf.WRITE_ONLY, A.nbytes)

In [ ]:
%%cl_kernel

__kernel void matmul(__global float* A, __global float* B, __global float* C, int N) {
    float sum = 0;
    int2 global_id = (int2)(get_global_id(0), get_global_id(1));

    if (global_id.x >= N || global_id.y >= N) {
        return;
    }
    
    // your code goes here
    
    for (int i = 0; i < N; i++) {
        int aij = global_id.y * N + i;
        int bij = i * N + global_id.x;
        sum  += A[aij] * B[bij];
    }

    int cij = global_id.y * N + global_id.x;
    C[cij] = sum;    
}

In [ ]:
# Launch kernel
block_size = (8, 8)
global_size = (A.shape[0], A.shape[1])
print(f'Launching with global_size={global_size}, block_size={block_size}')
n_warmup = 2
n_iters = 50

_ = profile_gpu(matmul, 20,
            queue, 
            global_size, 
            block_size,
            d_a,
            d_b,
            d_c,
            np.int32(N)
            )

## Verify Results

After running the kernel, copy the result back to the host and display it.

Refer to the [solution](matmul_global_solution.c) if you get stuck.

In [ ]:
# Copy result from GPU and display
C = np.empty_like(A)
cl.enqueue_copy(queue, C, d_c)
c_numpy = np.matmul(A, B)
#print("Result matrix C (A x B):", C)
np.testing.assert_almost_equal(C, c_numpy, decimal=3)
# Note: You need to implement the kernel for correct results!

# Shared memory caching

Your next task is to optimize the kernel execution time. One of the common practices in matrix multiplication is to optimize memory accesses. Elements in each input matrices A and B are accessed N times by reaching to global memory. While we can expect some level of caching happening under the hood, we can also cache parts of A and B matrices in local memory (OpenCL's __local).

Below is a helper function that will launch your new kernel. It will:
* validate the sizes of your matrices
* automatically allocate dynamic local memory for matrices A and B
* transfer matrices from host to device and back
* launch your kernel
* verify correctness of the calculations against numpy implementation
* if the results are correct it will launch the kernel multiple times to measure execution times

In [ ]:
def gpu_matrix_multiply(A, B, kernel, block_size=(16, 16), warmup=2, iters=100, *args):
    M, N_A = A.shape
    N_B, P = B.shape
    print(f"A: MxN ({M}, {N_A}), B: NxP ({N_B}, {P}), C: MxP ({M}, {P})")
    assert N_A == N_B, 'Inner matrix dimensions must match'

    mf = cl.mem_flags
    d_a = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=A)
    d_b = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=B)
    d_c = cl.Buffer(ctx, mf.WRITE_ONLY, M * P * 4)

    global_size = (P, M)
    local_size = block_size
    shared_mem_bytes = block_size[0] * block_size[1] * 4 #if use_local else 0
    d_slm_a = cl.LocalMemory(shared_mem_bytes)
    d_slm_b = cl.LocalMemory(shared_mem_bytes)
    def launch():
        kernel(queue, global_size, local_size, d_a, d_b, d_c, d_slm_a, d_slm_b, *args)

    print(f'Launching with global_size={global_size}, local_size={local_size}, 2 x shared_mem_bytes={2 * shared_mem_bytes}')

    launch()

    C = np.empty((M, P), dtype=np.float32)
    cl.enqueue_copy(queue, C, d_c)

    ref = np.matmul(A, B)
    np.testing.assert_almost_equal(C, ref, decimal=3)

    _ = profile_gpu(kernel, iters, queue, global_size, local_size, d_a, d_b, d_c, d_slm_a, d_slm_b, *args)

## Optimization task

Write an optimized kernel which will cache blocks from A and B matrices in local memory:
- for simplicity assume that the shapes of all matrices are multiple of block size
- make sure there are not data races - so synchronize local accesses

Implement the following algorithm:
- for each cache block
    - load 8x8 blocks from global memory into declared local memory
        - the main difficulty lays in accessing global memory based while iterating over cached blocks
    - produce partial matrix multiplication sum from data cached in local memory and accumulate in local variable
        - here you just need local ids
- dump accumulated sum into global memory C

In [ ]:
%%cl_kernel

#define THREAD_INDEX (get_local_id(1) * get_local_size(0) + get_local_id(0))

__kernel void shared_mem_kernel(__global float *A,
                     __global float *B,
                     __global float *C, 
                    __local float* slm_A, 
                    __local float* slm_B,
                    int N)
{
    float sum = 0.0f;
    int2 global_id = (int2)(get_group_id(0) * get_local_size(0) + get_local_id(0),
                            get_group_id(1) * get_local_size(1) + get_local_id(1));

    for (int b = 0; b < get_num_groups(0); b++) {
        int2 gidA = (int2)(b * get_local_size(0) + get_local_id(0), global_id.y);
        int2 gidB = (int2)(global_id.x, b * get_local_size(1) + get_local_id(1));

        slm_A[THREAD_INDEX] = A[gidA.y * N + gidA.x];
        slm_B[THREAD_INDEX] = B[gidB.y * N + gidB.x];

        barrier(CLK_LOCAL_MEM_FENCE);

        for (int i = 0; i < get_local_size(0); i++) {
            sum += slm_A[get_local_size(0) * get_local_id(1) + i] *
                   slm_B[get_local_size(0) * i + get_local_id(0)];
        }

        barrier(CLK_LOCAL_MEM_FENCE);
    }

    C[global_id.y * N + global_id.x] = sum;
}

Execute the kernel with SLM.

In [ ]:
args = (np.int32(N),)
gpu_matrix_multiply(A, B, shared_mem_kernel, (16, 16), 2, 100, *args)

Refer to the [solution](matmul_slm_solution.c) if you get stuck.

# Non-square matrices

Adapt your solution to work with non-square matrices. You will need to:
* pass sizes of matrices to your kernel - as they are non-square so can have different shapes than only N.
* Use these shapes in your kernel to access elements.
* think about iterating through consecutive cached blocks in the for-loop. Make sure to look iterate from the perspective of output matrix.

In [ ]:
%%cl_kernel

#define THREAD_INDEX (get_local_id(1) * get_local_size(0) + get_local_id(0))

__kernel void non_square_matmul(__global float *A,
                                __global float *B,
                                __global float *C, 
                                __local float* slm_A, 
                                __local float* slm_B,
                                int M,
                                int N,
                                int P)
{
    float sum = 0.0f;
    int2 global_id = (int2)(get_group_id(0) * get_local_size(0) + get_local_id(0),
                            get_group_id(1) * get_local_size(1) + get_local_id(1));

    int nr_blocks = (N + get_local_size(0) - 1) / get_local_size(0);
        
    for (int b = 0; b < nr_blocks; b++) {
        int2 gidA = (int2)(b * get_local_size(0) + get_local_id(0), global_id.y);
        int2 gidB = (int2)(global_id.x, b * get_local_size(1) + get_local_id(1));

        slm_A[THREAD_INDEX] = A[gidA.y * N + gidA.x];
        slm_B[THREAD_INDEX] = B[gidB.y * P + gidB.x];

        barrier(CLK_LOCAL_MEM_FENCE);

        for (int i = 0; i < get_local_size(0); i++) {
            sum += slm_A[get_local_size(0) * get_local_id(1) + i] *
                   slm_B[get_local_size(0) * i + get_local_id(0)];
        }

        barrier(CLK_LOCAL_MEM_FENCE);
    }

    C[global_id.y * P + global_id.x] = sum;
}

Execute

In [ ]:
A_ = np.random.randn(512, 1024).astype(np.float32)
B_ = np.random.randn(1024, 768).astype(np.float32)
M, N, P = A_.shape[0], A_.shape[1], B_.shape[1]
args = (np.int32(M), np.int32(N), np.int32(P))
gpu_matrix_multiply(A_, B_, non_square_matmul, (16, 16), 2, 100, *args)

Refer to the [solution](matmul_solution_non_square.c) if you get stuck.